In [17]:
import xarray as xr
import matplotlib.pylab as plt
import xesmf as xe
import geopandas 
import rioxarray

In [19]:
in_dir = "/data/ABOVE/MODIS/LAI_CLM_DOMAIN/"
out_dir = ("/data/ABOVE/MODIS/LAI_CLM_DOMAIN/processed/")

In [20]:
clm_image = xr.open_dataset(in_dir+"freeCLM5.1forILAMB.nc")
clm_image = clm_image["TLAI"].isel(time=1)


In [21]:
lst = xr.open_dataarray("/data/ABOVE/Final_data/LST_Final/LST/Annual_Mean/geographic/lst_mean_annual.nc")
# albedo = albedo.rename({"x":"lon","y":"lat"})

In [22]:
ds_out = xr.Dataset({
    'lat': (['lat'], clm_image['lat'].values),
    'lon': (['lon'], clm_image['lon'].values)
})
ds_in = xr.Dataset({
    'lat': (['lat'], lst['lat'].values),
    'lon': (['lon'], lst['lon'].values)
})

In [8]:
regridder = xe.Regridder(ds_in, ds_out, 'bilinear')
# regridder.to_netcdf(out_dir+"regridder_albedo_entire_clm_domain.nc")

In [23]:
lst_regrided = regridder(lst)

/data/home/hamiddashti/miniconda3/envs/conda_env/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


In [25]:
lst_regrided = lst_regrided.rio.write_crs(4326)
lst_regrided = lst_regrided.rename({"lon":"x","lat":"y"})
geodf = geopandas.read_file("/data/ABOVE/Final_data/shp_files/CoreDomain_geographic.shp")
clipped = lst_regrided.rio.clip(geodf.geometry.values, geodf.crs)

In [27]:
clipped.to_netcdf(out_dir+"lst_mean_annual.nc")
